# **Lab1: Regression**
In *lab 1*, you need to finish:

1.  Basic Part: Implement the regression model to predict people's grip force from their weight.
You can use either Matrix Inversion or Gradient Descent.


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implementing a regression model to predict grip force in a different way (for example, with more variables) than the basic part




---
# 1. Basic Part (50%)
In the first part, you need to implement the regression to predict grip force

Please save the prediction result in a CSV file and submit it to Kaggle

### Import Packages

> Note: You **cannot** import any other package


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

### Global attributes
Define the global attributes\
You can also add your own global attributes here

In [3]:
training_dataroot = 'lab1_basic_training.csv' # Training data file file named as 'lab1_basic_training.csv'
testing_dataroot = 'lab1_basic_testing.csv'   # Testing data file named as 'lab1_basic_testing.csv'
output_dataroot = 'lab1_basic.csv' # Output file will be named as 'lab1_basic.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 100 elements

### Load the Input File
First, load the basic input file **lab1_basic_training.csv** and **lab1_basic_testing.csv**

Input data would be stored in *training_datalist* and *testing_datalist*

In [4]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

### Implement the Regression Model

> Note: It is recommended to use the functions we defined, you can also define your own functions

#### Step 1: Split Data
Split data in *training_datalist* into training dataset and validation dataset


In [6]:
def SplitData(data, split_ratio):
    """
    Splits the given dataset into training and validation sets based on the specified split ratio.

    Parameters:
    - data (numpy.ndarray): The dataset to be split. It is expected to be a 2D array where each row represents a data point and each column represents a feature.
    - split_ratio (float): The ratio of the data to be used for training. For example, a value of 0.8 means 80% of the data will be used for training and the remaining 20% for validation.

    Returns:
    - training_data (numpy.ndarray): The portion of the dataset used for training.
    - validation_data (numpy.ndarray): The portion of the dataset used for validation.

    """
    training_data = []
    validation_data = []

    # TODO
    train_size = int(len(data) * split_ratio)
    training_data = data[:train_size]
    validation_data = data[train_size:]

    return training_data, validation_data



#### Step 2: Preprocess Data
Handle unreasonable data and missing data

> Hint 1: Outliers and missing data can be addressed by either removing them or replacing them using statistical methods (e.g., the mean of all data).

> Hint 2: Missing data are represented as `np.nan`, so functions like `np.isnan()` can be used to detect them.

> Hint 3: Methods such as the Interquartile Range (IQR) can help detect outliers

In [7]:
def PreprocessData(data):
    """
    Preprocess the given dataset and return the result.

    Parameters:
    - data (numpy.ndarray): The dataset to preprocess. It is expected to be a 2D array where each row represents a data point and each column represents a feature.

    Returns:
    - preprocessedData (numpy.ndarray): Preprocessed data.
    """
    preprocessedData = []

    # TODO
    df = pd.DataFrame(data)
    df = df.fillna(df.median())
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]
    preprocessedData = df.to_numpy()
    return preprocessedData


### Step 3: Implement Regression
You have to use Gradient Descent to finish this part

In [8]:
def Regression(dataset):
    """
    Performs regression on the given dataset and return the coefficients.

    Parameters:
    - dataset (numpy.ndarray): A 2D array where each row represents a data point.

    Returns:
    - w (numpy.ndarray): The coefficients of the regression model. For example, y = w[0] + w[1] * x + w[2] * x^2 + ...
    """

    X = dataset[:, :1]
    y = dataset[:, 1]

    # TODO: Decide on the degree of the polynomial
    degree = 2  # For example, quadratic regression

    # Add polynomial features to X
    X_poly = np.ones((X.shape[0], 1))  # Add intercept term (column of ones)
    for d in range(1, degree + 1):
        X_poly = np.hstack((X_poly, X ** d))  # Add x^d terms to feature matrix

    # Initialize coefficients (weights) to zero
    num_dimensions = X_poly.shape[1]  # Number of features (including intercept and polynomial terms)
    w = np.zeros(num_dimensions)

    # TODO: Set hyperparameters
    num_iteration = 100000
    learning_rate = 0.000000083035
    # Gradient Descent
    m = len(y)  # Number of data points
    for iteration in range(num_iteration):
        # TODO: Prediction using current weights and compute error
        y_pred = X_poly.dot(w)
        error = y_pred - y
        # TODO: Compute gradient

        gradient = (1/m) * X_poly.T.dot(error)
        # TODO: Update the weights
        w -= learning_rate * gradient
        # TODO: Optionally, print the cost every 100 iterations
        if iteration % 100 == 0:
            cost = (1/(2*m)) * np.sum(error ** 2)
            print(f"Iteration {iteration}, Cost: {cost}")
    return w


### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*

In [12]:

def MakePrediction(w, test_dataset):
    """
    Predicts the output for a given test dataset using a regression model.

    Parameters:
    - w (numpy.ndarray): The coefficients of the model, where each element corresponds to
                               a coefficient for the respective power of the independent variable.
    - test_dataset (numpy.ndarray): A 1D array containing the input values (independent variable)
                                          for which predictions are to be made.

    Returns:
    - list/numpy.ndarray: A list or 1d array of predicted values corresponding to each input value in the test dataset.
    """
    prediction = []

    # TODO
    X_test = test_dataset[:, 0].reshape(-1, 1)
    degree = 2
    X_poly_test = np.ones((X_test.shape[0], 1))
    for d in range(1, degree + 1):
        X_poly_test = np.hstack((X_poly_test, X_test ** d))

    prediction = X_poly_test.dot(w)
    return prediction


### Step 5: Train Model and Generate Result

Use the above functions to train your model on training dataset, and predict the answer of testing dataset.

Save your predicted values in `output_datalist`

> Notice: **Remember to inclue the coefficients of your model in the report**



In [13]:
# (1) Split data
training_data, validation_data = SplitData(training_datalist, 0.8)
# (2) Preprocess data
training_data = PreprocessData(training_data)
validation_data = PreprocessData(validation_data)
# (3) Train regression model
w = Regression(training_data)
# (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth
validation_pred = MakePrediction(w, validation_data)
y_true = validation_data[:, 1]
MAPE = np.mean(np.abs((y_true - validation_pred) / y_true)) * 100
print(f"Validation MAPE: {MAPE}%")
# (5) Make prediction of testing dataset and store the values in output_datalist
testing_data = PreprocessData(testing_datalist)
output_datalist = MakePrediction(w, testing_data)

Iteration 0, Cost: 1057.6937412194436
Iteration 100, Cost: 1032.5216452752306
Iteration 200, Cost: 1007.9834627642224
Iteration 300, Cost: 984.0632077298128
Iteration 400, Cost: 960.7452973913894
Iteration 500, Cost: 938.014541975896
Iteration 600, Cost: 915.8561348059652
Iteration 700, Cost: 894.2556426377388
Iteration 800, Cost: 873.1989962426571
Iteration 900, Cost: 852.6724812266391
Iteration 1000, Cost: 832.6627290808957
Iteration 1100, Cost: 813.1567084583681
Iteration 1200, Cost: 794.1417166702798
Iteration 1300, Cost: 775.6053713970135
Iteration 1400, Cost: 757.5356026081871
Iteration 1500, Cost: 739.9206446864436
Iteration 1600, Cost: 722.7490287498755
Iteration 1700, Cost: 706.0095751681288
Iteration 1800, Cost: 689.6913862672017
Iteration 1900, Cost: 673.7838392183203
Iteration 2000, Cost: 658.2765791060667
Iteration 2100, Cost: 643.1595121713591
Iteration 2200, Cost: 628.4227992249413
Iteration 2300, Cost: 614.0568492268351
Iteration 2400, Cost: 600.0523130278588
Iteration 

### *Write the Output File*

Write the prediction to output csv and upload the file to Kaggle
> Format: 'Id', 'gripForce'


In [15]:
# Assume that output_datalist is a list (or 1d array) with length = 100

with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Id', 'gripForce'])
  for i in range(len(output_datalist)):
    writer.writerow([i,output_datalist[i]])


# 2. Advanced Part (45%)
In the second part, you need to implement regression differently from the basic part to improve your grip force predictions. You must use more than two features.

You can choose either matrix inversion or gradient descent for this part

We have provided `lab1_advanced_training.csv` for your training

> Notice: Be cautious of the "gender" attribute, as it is represented by "F"/"M" rather than a numerical value.

Please save the prediction result in a CSV file and submit it to Kaggle

In [16]:
training_dataroot = 'lab1_advanced_training.csv' # Training data file file named as 'lab1_advanced_training.csv'
testing_dataroot = 'lab1_advanced_testing.csv'   # Testing data file named as 'lab1_advanced_testing.csv'
output_dataroot = 'lab1_advanced.csv' # Output file will be named as 'lab1_advanced.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 3000 elements

In [17]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

In [61]:
def SplitData(data, split_ratio):
    np.random.shuffle(data)
    train_size = int(len(data) * split_ratio)
    training_data = data[:train_size]
    validation_data = data[train_size:]
    return training_data, validation_data

def PreprocessData(data, is_test=False):
    if is_test:
        df = pd.DataFrame(data, columns=['age', 'gender', 'height', 'weight', 'bodyFat', 'diastolic', 'systolic'])
    else:
        df = pd.DataFrame(data, columns=['age', 'gender', 'height', 'weight', 'bodyFat', 'diastolic', 'systolic', 'gripForce'])
    df['gender'] = df['gender'].map({'F': 5, 'M': 7})
    df = df.dropna()
    df = df.astype('float64')
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    if is_test == False :df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

    return df.to_numpy()

def Regression(dataset):
    X = dataset[:, :-1]
    y = dataset[:, -1]
    num_dimensions = X.shape[1]
    w = np.zeros(num_dimensions)
    num_iteration = 1000000
    learning_rate = 0.000033
    m = len(y)

    for iteration in range(num_iteration):
        y_pred = X.dot(w) 
        error = y_pred - y
        gradient = (1/m) * X.T.dot(error)
        w -= learning_rate * gradient
        if iteration % 100000 == 0:
            cost = (1/(2*m)) * np.sum(error ** 2)
            print(f"Iteration {iteration}, Cost: {cost}")
    return w

def MakePrediction(w, test_dataset):
    X_test = test_dataset
    prediction = X_test.dot(w)
    return prediction

training_data, validation_data = SplitData(training_datalist, 0.8)

training_data = PreprocessData(training_data, is_test=False)
validation_data = PreprocessData(validation_data, is_test=False)

w = Regression(training_data)

validation_features = validation_data[:, :-1]
validation_true = validation_data[:, -1]
validation_pred = MakePrediction(w, validation_features)
non_zero_y_true = np.where(validation_true == 0, 1e-6, validation_true)
MAPE = np.mean(np.abs((validation_true - validation_pred) / non_zero_y_true)) * 100
print(f"Validation MAPE: {MAPE}%")
testing_data = PreprocessData(testing_datalist, is_test=True)
output_datalist = MakePrediction(w, testing_data)+np.mean(validation_true-validation_pred)
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'gripForce'])
    for i in range(len(output_datalist)):
        writer.writerow([i, output_datalist[i]])


Iteration 0, Cost: 1059.7533645349708
Iteration 100000, Cost: 20.63643233925237
Iteration 200000, Cost: 20.345306297892378
Iteration 300000, Cost: 20.290160995027698
Iteration 400000, Cost: 20.279715332338668
Iteration 500000, Cost: 20.277736707445076
Iteration 600000, Cost: 20.277361914903317
Iteration 700000, Cost: 20.277290921432144
Iteration 800000, Cost: 20.277277473798083
Iteration 900000, Cost: 20.27727492653763
Validation MAPE: 11.427412977078653%


# Save the Code File
Please save your code and submit it as an ipynb file! (**Lab1.ipynb**)